# LangChain Expression Language (LCEL)

In [10]:
import os
from Models import GroqChatLLM

os.system("clear")

llm = GroqChatLLM()
print(llm.invoke("hello").content)

Hello. How can I assist you today?


In [11]:
%%script true
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [12]:
#!pip install pydantic==1.10.8

In [13]:
from langchain.prompts import ChatPromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [14]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = llm
output_parser = StrOutputParser()

In [15]:
chain = prompt | model | output_parser

In [16]:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [17]:
#from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [18]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    #embedding=OpenAIEmbeddings()
    embedding=OllamaEmbeddings(model="nomic-embed-text")
    )
retriever = vectorstore.as_retriever()

/home/tjamil/Desktop/Agents/.conda/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [19]:
retriever.get_relevant_documents("where did harrison work?")

/tmp/ipykernel_24746/3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [20]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [21]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [22]:
from langchain.schema.runnable import RunnableMap

In [23]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [24]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [25]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [26]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [27]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = GroqChatLLM(temperature=0).bind(functions=functions)

In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_c0nn', 'function': {'arguments': '{"airport_code": "SFO"}', 'name': 'weather_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 233, 'total_tokens': 250, 'completion_time': 0.068607588, 'prompt_time': 0.064967729, 'queue_time': 0.005420363999999997, 'total_time': 0.133575317}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8f50d708-ab13-461b-a54c-f7da2e05b294-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'SFO'}, 'id': 'call_c0nn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 233, 'output_tokens': 17, 'total_tokens': 250})

In [31]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [32]:
model = model.bind(functions=functions)

In [33]:
runnable = prompt | model

In [34]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dk1f', 'function': {'arguments': '{"team_name": "patriots"}', 'name': 'sports_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 299, 'total_tokens': 317, 'completion_time': 0.072, 'prompt_time': 0.08025644, 'queue_time': 0.005012671999999996, 'total_time': 0.15225644}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f5c673b9-f2aa-40e0-af5a-bb9b123ca0ba-0', tool_calls=[{'name': 'sports_search', 'args': {'team_name': 'patriots'}, 'id': 'call_dk1f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 299, 'output_tokens': 18, 'total_tokens': 317})

## Interface

In [35]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = llm#ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [36]:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.'

In [37]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.',
 'Why did the frog go to the doctor? \n\nBecause it had a "ribbiting" cough.']

In [38]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 go
 to
 the
 doctor
?
 


Because
 it
 had
 a
 gr
izzly
 cough
.



In [39]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.'